# Collect dataset for incision detection

We will collect jpegs of incision from application directory. This data will be uploaded to CVAT and annotated.

In [32]:
import os
from pathlib import Path
import shutil
from tqdm import tqdm
import cv2
import itertools

DATA_BASEDIR = Path(
    os.getenv(
        # "DATA_BASEDIR", r"~/mnt/pole/storage/plzen4-ntis/projects/korpusy_cv/pigleg_surgery/"
        "DATA_BASEDIR", r"."
    )
).expanduser()

APP_DATA_BASEDIR = Path(
    os.getenv(
        "APP_DATA_BASEDIR", r"/webapps/piglegsurgery/piglegsurgeryweb/media_symlink/"
    )
)


In [33]:
assert DATA_BASEDIR.exists()

In [34]:
input_dir = APP_DATA_BASEDIR/ "upload"
output_dir = DATA_BASEDIR / "incision_detection_march2023"

In [35]:
def get_frame_to_process(filename):
    
    if Path(filename).suffix.lower() in (".png", ".jpg", ".jpeg", ".tif", ".tiff"):
        # image
        img = cv2.imread(filename)
    else:
        cap = cv2.VideoCapture(str(filename))
        last_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
        # print(last_frame)
        cap.set(cv2.CAP_PROP_POS_FRAMES, last_frame)
        ret, img = cap.read()
        i = 0
        while (not ret) and (i < 50) and (i < last_frame - 1):
            # print('Last frame capture error, frame', last_frame - i)
            cap.set(cv2.CAP_PROP_POS_FRAMES, last_frame - i - 1)
            ret, img = cap.read()
            i += 1
        cap.release()
        if not ret:
            return None
    return img

In [36]:

output_dir.mkdir(parents=True, exist_ok=True)
iterator = itertools.chain(
    input_dir.glob("*/*.*"),
    # input_dir.glob("**/*.mov"),
    # input_dir.glob("**/*.mp4")
    # input_dir.glob("**/*.webm")
)

hashes = {}
none_files = []
for fn in tqdm(list(iterator)):
    if fn.suffix.lower() in (".avi", ".mov", ".mp4", ".webm"):
        frame = get_frame_to_process(fn)
        # frame.flags.writeable = False
        if frame is not None:
            h = hash(frame.tobytes())
            if h in hashes:
                # fn_out = output_dir / ("_" + hashes[h] + ".duplicate." + fn.parent.name + ".jpg")
                # cv2.imwrite(str(fn_out), frame)
                pass
            else:
                hashes[h] = fn.parent.name + ".jpg"
                fn_out = output_dir / (fn.parent.name + ".jpg")
                cv2.imwrite(str(fn_out), frame)
            
        else:
            none_files.append(fn)
        # print(fn_out)
        
        # else: 
        #     pass
print(f"Failed to read frame from {len(none_files)} files:")
print(none_files) 


  6%|█████▊                                                                                                | 53/929 [00:07<01:11, 12.32it/s][h264 @ 0x560d38c06940] A non-intra slice in an IDR NAL unit.
[h264 @ 0x560d38c06940] decode_slice_header error
[h264 @ 0x560d38cca840] A non-intra slice in an IDR NAL unit.
[h264 @ 0x560d38cca840] decode_slice_header error
 10%|█████████▊                                                                                            | 89/929 [00:10<01:15, 11.12it/s][h264 @ 0x560d38c26c80] co located POCs unavailable
[h264 @ 0x560d38beed40] reference picture missing during reorder
[h264 @ 0x560d38beed40] Missing reference picture, default is 4
[h264 @ 0x560d38c97c00] mmco: unref short failure
[h264 @ 0x560d38c26c80] co located POCs unavailable
[h264 @ 0x560d38c97c00] mmco: unref short failure
[h264 @ 0x560d38c26c80] co located POCs unavailable
[h264 @ 0x560d38c97c00] mmco: unref short failure
[h264 @ 0x560d38c26c80] co located POCs unavailable
[h264 @ 0

Failed to read frame from these files:
[PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media_symlink/upload/20211110-115322_Einzelknopfnaht_ed77d96e44003d0efbea3e8bfb062b24fbf30d2a/output.mp4'), PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media_symlink/upload/20211119-150757_pigleg_test_eeccc81df26ed500c9ffb0a5055aa453b9f92365/output.mp4'), PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media_symlink/upload/20211012-185853_12.10.2021_B_Glasl,Johanna_Einzelknopfnaht_b7480c505f23dd7692cd7989074f3630d89d9658/12.10.2021_B_GlaslJohanna_Einzelknopfnaht.webm'), PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media_symlink/upload/20211208-124317_output_452778d7de00991dfd6486600b1d9b61939f3d4f/output.mp4'), PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media_symlink/upload/20220609-181735_c94b769a-c358-4710-b9ba-7e6005b2ba6c_f42140af0dde72a3e92f2020593f365b62705c37/c94b769a-c358-4710-b9ba-7e6005b2ba6c.mp4'), PosixPath('/webapps/piglegsurgery/piglegsurgeryweb/media_symlink

In [37]:
len(none_files)

25